# Final Project Overview
Your boss asked you to compile the list of the top 10 largest banks in the world ranked by market capitalization in billion USD. Further, you need to transform the data and store it in USD, GBP, EUR, and INR per the exchange rate information made available to you as a CSV file. You should save the processed information table locally in a CSV format and as a database table. Managers from different countries will query the database table to extract the list and note the market capitalization value in their own currency.

## Direction
1. Write a function to extract the tabular information from the given URL under the heading By Market Capitalization, and save it to a data frame.
2. Write a function to transform the data frame by adding columns for Market Capitalization in GBP, EUR, and INR, rounded to 2 decimal places, based on the exchange rate information shared as a CSV file.
3. Write a function to load the transformed data frame to an output CSV file.
4. Write a function to load the transformed data frame to an SQL database server as a table.
5. Write a function to run queries on the database table.
6. Run the following queries on the database table:
   - Extract the information for the London office, that is Name and MC_GBP_Billion
   - Extract the information for the Berlin office, that is Name and MC_EUR_Billion
   - Extract the information for New Delhi office, that is Name and MC_INR_Billion
8. Write a function to log the progress of the code.
9. While executing the data initialization commands and function calls, maintain appropriate log entries.

importing packages/libraries

In [107]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
from datetime import datetime
import sqlite3

Creating a log function

In [99]:
def log_info(data):
    file = 'code_log.txt'
    timeStamp_format = '%Y-%m-%d %H:%M:%S'
    timeStamp = datetime.now().strftime(timestamp_format)
    with open(file, 'a+') as f:
        f.write(f'{timeStamp} - {data}\n')

Accessing URL data

In [100]:
url = 'https://web.archive.org/web/20230908091635 /https://en.wikipedia.org/wiki/List_of_largest_banks'
response = requests.get(url)
log_info(f'URL accessed with confirmation code {response.status_code}')

Parsing HTML page

In [101]:
soup = BeautifulSoup(response.content, 'html.parser')

Fetching data from parsed data (soup)

In [102]:
def extract(soup):
    table = soup.find_all('table')[0]
    rows = table.find_all('tr')
    rows = rows[1:]
    data = {}
    data['rank'] = []
    data['bank_name'] = []
    data['market_cap(USD_billion)'] = []
    for row in rows:
        rank = int(row.find_all('td')[0].text.strip())
        bank_name = row.find_all('td')[1].find_all('a')[-1].text
        market_cap = float(row.find_all('td')[2].text.strip())
        data['rank'].append(rank)
        data['bank_name'].append(bank_name)
        data['market_cap(USD_billion)'].append(market_cap)
    return data

In [106]:
data = extract(soup)
df = pd.DataFrame(data)
log_info('data extracted and loaded into a DataFrame')

In [104]:
df

,rank,bank_name,market_cap(USD_billion)
0,1,JPMorgan Chase,432.92
1,2,Bank of America,231.52
2,3,Industrial and Commercial Bank of China,194.56
3,4,Agricultural Bank of China,160.68
4,5,HDFC Bank,157.91
5,6,Wells Fargo,155.87
6,7,HSBC Holdings PLC,148.90
7,8,Morgan Stanley,140.83
8,9,China Construction Bank,139.82
9,10,Bank of China,136.81
